# Call Graph Analysis and SCC Detection

This notebook explores UnifyWeaver's advanced code analysis features:

- **Call Graph Construction** - Building dependency graphs from Prolog code
- **SCC Detection** - Finding Strongly Connected Components (mutual recursion)
- **Pattern Analysis** - Understanding recursion patterns
- **Dependency Visualization** - Visualizing predicate relationships

## Learning Objectives

- Understand how UnifyWeaver analyzes code structure
- Build and inspect call graphs
- Detect mutual recursion using Tarjan's algorithm
- Visualize code dependencies

## Setup

Load UnifyWeaver and analysis modules.

In [ ]:
% Load initialization
['../init'].

% Load analysis modules
use_module(unifyweaver(core/advanced/call_graph)).
use_module(unifyweaver(core/advanced/scc_detection)).
use_module(unifyweaver(core/advanced/pattern_matchers)).

## Example 1: Simple Call Graph

Let's start with a simple predicate and build its call graph.

In [ ]:
% Define ancestor predicate
:- dynamic ancestor/2.
:- dynamic parent/2.

% Parent facts
parent(abraham, isaac).
parent(isaac, jacob).

% Ancestor rules
ancestor(X, Y) :- parent(X, Y).
ancestor(X, Z) :- parent(X, Y), ancestor(Y, Z).

### Build Call Graph

In [ ]:
% Build call graph for ancestor
build_call_graph([ancestor/2], Graph),
format('Call Graph for ancestor/2:~n'),
forall(member(Edge, Graph),
    format('  ~w~n', [Edge])).

### Analyze Dependencies

In [ ]:
% Get all dependencies of ancestor/2
get_dependencies(ancestor/2, Deps),
format('Dependencies of ancestor/2: ~w~n', [Deps]).

% Check if self-recursive
(is_self_recursive(ancestor/2) ->
    writeln('✓ ancestor/2 is self-recursive')
;
    writeln('✗ ancestor/2 is not self-recursive')
).

## Example 2: Mutual Recursion Detection

Now let's detect mutual recursion with the even/odd example.

In [ ]:
% Define mutually recursive predicates
:- dynamic is_even/1.
:- dynamic is_odd/1.

is_even(0).
is_even(N) :- N > 0, N1 is N - 1, is_odd(N1).

is_odd(1).
is_odd(N) :- N > 1, N1 is N - 1, is_even(N1).

### Build Call Graph for Both Predicates

In [ ]:
% Build call graph for both predicates
build_call_graph([is_even/1, is_odd/1], Graph),
format('Call Graph for is_even/1 and is_odd/1:~n'),
forall(member(Edge, Graph),
    format('  ~w~n', [Edge])).

### Find Strongly Connected Components (SCCs)

In [ ]:
% Find SCCs using Tarjan's algorithm
find_sccs(Graph, SCCs),
format('Strongly Connected Components:~n'),
forall(member(SCC, SCCs),
    format('  ~w~n', [SCC])).

### Check if SCC is Trivial

In [ ]:
% Check each SCC
forall(member(SCC, SCCs),
    (   is_trivial_scc(SCC) ->
        format('  ~w is trivial (single predicate)~n', [SCC])
    ;
        format('  ~w is NON-TRIVIAL (mutual recursion!)~n', [SCC])
    )
).

## Example 3: Complex Call Graph

Let's analyze a more complex system with multiple predicates.

In [ ]:
% Define a small program with multiple predicates
:- dynamic grandparent/2.
:- dynamic sibling/2.
:- dynamic cousin/2.

% grandparent uses parent
grandparent(X, Z) :- parent(X, Y), parent(Y, Z).

% sibling: same parent, different children
sibling(X, Y) :- parent(P, X), parent(P, Y), X \= Y.

% cousin: parents are siblings
cousin(X, Y) :- parent(P1, X), parent(P2, Y), sibling(P1, P2).

### Build Complete Call Graph

In [ ]:
% Build call graph for all predicates
build_call_graph([grandparent/2, sibling/2, cousin/2], Graph),
format('Complete Call Graph:~n'),
forall(member(Edge, Graph),
    format('  ~w~n', [Edge])).

### Find Predicate Groups

Find all predicates reachable from a starting predicate.

In [ ]:
% Find all predicates in the group starting from cousin/2
predicates_in_group(cousin/2, Group),
format('All predicates reachable from cousin/2: ~w~n', [Group]).

## Example 4: Pattern Detection

Let's use pattern matchers to analyze recursion types.

In [ ]:
% Define various recursive patterns
:- dynamic count/3.     % Tail recursive
:- dynamic factorial/2. % Linear recursive
:- dynamic fib/2.       % Tree recursive (or linear if detected)

% Tail recursive count
count([], Acc, Acc).
count([_|T], Acc, N) :-
    Acc1 is Acc + 1,
    count(T, Acc1, N).

% Linear recursive factorial
factorial(0, 1).
factorial(N, F) :-
    N > 0,
    N1 is N - 1,
    factorial(N1, F1),
    F is N * F1.

% Fibonacci (can be detected as linear or tree)
fib(0, 0).
fib(1, 1).
fib(N, F) :-
    N > 1,
    N1 is N - 1,
    N2 is N - 2,
    fib(N1, F1),
    fib(N2, F2),
    F is F1 + F2.

### Detect Tail Recursion

In [ ]:
% Check if count/3 is tail recursive
(is_tail_recursive_accumulator(count/3, AccInfo) ->
    format('✓ count/3 is tail recursive: ~w~n', [AccInfo])
;
    writeln('✗ count/3 is not tail recursive')
).

### Detect Linear Recursion

In [ ]:
% Check if factorial/2 is linear recursive
(is_linear_recursive_streamable(factorial/2) ->
    writeln('✓ factorial/2 is linear recursive')
;
    writeln('✗ factorial/2 is not linear recursive')
).

### Count Recursive Calls

In [ ]:
% Count recursive calls in fibonacci
functor(FibHead, fib, 2),
user:clause(FibHead, FibBody),
contains_call_to(FibBody, fib),
count_recursive_calls(FibBody, fib, Count),
format('Fibonacci body has ~w recursive calls~n', [Count]).

## Visualization with DOT Format

Let's generate a Graphviz DOT representation of our call graph.

In [ ]:
% Helper to generate DOT format
generate_dot(Graph, DotCode) :-
    findall(Line,
        (   member(From -> To, Graph),
            format(atom(Line), '  "~w" -> "~w";', [From, To])
        ),
        Lines),
    atomic_list_concat(['digraph CallGraph {', '  rankdir=LR;' | Lines], '\n', Body),
    format(atom(DotCode), '~w~n}~n', [Body]).

% Generate DOT for even/odd graph
build_call_graph([is_even/1, is_odd/1], Graph),
generate_dot(Graph, DotCode),
writeln('DOT Code for even/odd call graph:'),
writeln(DotCode).

### Save DOT File

In [ ]:
% Save DOT code to file
open('../output/even_odd_graph.dot', write, Stream),
write(Stream, DotCode),
close(Stream),
writeln('✓ Saved to ../output/even_odd_graph.dot').

writeln('To visualize, run:'),
writeln('  dot -Tpng ../output/even_odd_graph.dot -o even_odd_graph.png').

## Exercise: Analyze Your Own Code

Try defining your own predicates and analyzing them!

In [ ]:
% Define your predicates here
% Then build call graphs, find SCCs, and detect patterns

% Example:
% :- dynamic my_predicate/2.
% my_predicate(...) :- ...

% build_call_graph([my_predicate/2], Graph).


## Summary

In this notebook, you learned:

✅ How to build call graphs from Prolog code

✅ How to detect Strongly Connected Components (SCCs) for mutual recursion

✅ How to use pattern matchers to classify recursion types

✅ How to analyze predicate dependencies

✅ How to visualize call graphs with DOT format

## Advanced Topics

For more advanced analysis:

- **Topological Ordering**: Use `topological_order/2` to order SCCs by dependencies
- **Custom Pattern Matchers**: Write your own pattern detection predicates
- **Accumulator Pattern Extraction**: Use `extract_accumulator_pattern/2` for detailed analysis
- **Forbid Linear Recursion**: Use `forbid_linear_recursion/1` to force different compilation strategies

## References

- Chapter 10: Prolog Introspection and Theory
- `src/unifyweaver/core/advanced/call_graph.pl`
- `src/unifyweaver/core/advanced/scc_detection.pl`
- `src/unifyweaver/core/advanced/pattern_matchers.pl`